In [1]:
import pandas
import pulp
import geopandas # standard library for geographical data in Python
import numpy # standard library for numerical and array-based computing
import spopt # library for spatial optimisation

In [2]:
postcodes = pandas.read_csv("../uk_postcodes.csv")

# spatialize the data
postcodes = geopandas.GeoDataFrame(
    postcodes, 
    geometry=geopandas.points_from_xy(postcodes.lon, postcodes.lat, crs="epsg:4326")
)

bristol_postcodes = postcodes[postcodes.outward.str.startswith("BS")].copy()

# is the postcode in the center of the city? 
is_inner_city = bristol_postcodes.outward.str.lstrip("BS").astype(int) < 10

# if so, keep it around
bristol_innercity = bristol_postcodes[is_inner_city].copy()

# convert the data to utm for distance calculations: 
bristol_innercity_utm = bristol_innercity.to_crs(
    bristol_innercity.estimate_utm_crs()
)
bristol_postcodes = bristol_postcodes.to_crs(
    bristol_postcodes.estimate_utm_crs()
)

# Do: fit your own coverage problems!

Now it is time for you to try your hand at solving coverage problems. 

## One again, with feeling

<div class="alert alert-warning">
    
Using the methods we discussed above, can you create a new LSCP problem object to cover all Bristol post codes using a 5000 meter service radius? How many facilities are needed, and in which postcode are they located?

</div>

In [3]:
lscp1 = spopt.locate.LSCP.from_geodataframe(
    gdf_demand=bristol_postcodes,
    gdf_fac=bristol_postcodes,
    service_radius = 5000,
    demand_col="geometry", 
    facility_col="geometry",
).solve(pulp.COIN_CMD(msg=False))

In [4]:
lscp1

In [5]:
lscp1.problem

lscp:
MINIMIZE
1*y_0_ + 1*y_10_ + 1*y_11_ + 1*y_12_ + 1*y_13_ + 1*y_14_ + 1*y_15_ + 1*y_16_ + 1*y_17_ + 1*y_18_ + 1*y_19_ + 1*y_1_ + 1*y_20_ + 1*y_21_ + 1*y_22_ + 1*y_23_ + 1*y_24_ + 1*y_25_ + 1*y_26_ + 1*y_27_ + 1*y_28_ + 1*y_29_ + 1*y_2_ + 1*y_30_ + 1*y_31_ + 1*y_32_ + 1*y_33_ + 1*y_34_ + 1*y_35_ + 1*y_36_ + 1*y_37_ + 1*y_38_ + 1*y_39_ + 1*y_3_ + 1*y_40_ + 1*y_41_ + 1*y_42_ + 1*y_4_ + 1*y_5_ + 1*y_6_ + 1*y_7_ + 1*y_8_ + 1*y_9_ + 0
SUBJECT TO
_C1: y_0_ + y_11_ + y_22_ + y_31_ + y_34_ + y_37_ + y_38_ + y_39_ + y_3_
 + y_41_ + y_42_ + y_4_ + y_8_ >= 1

_C2: y_1_ + y_26_ >= 1

_C3: y_2_ + y_42_ >= 1

_C4: y_0_ + y_38_ + y_39_ + y_3_ + y_42_ + y_8_ >= 1

_C5: y_0_ + y_11_ + y_22_ + y_31_ + y_34_ + y_37_ + y_38_ + y_41_ + y_42_
 + y_4_ + y_8_ >= 1

_C6: y_24_ + y_30_ + y_31_ + y_5_ + y_9_ >= 1

_C7: y_23_ + y_28_ + y_6_ + y_7_ >= 1

_C8: y_23_ + y_28_ + y_38_ + y_39_ + y_6_ + y_7_ + y_8_ >= 1

_C9: y_0_ + y_11_ + y_31_ + y_37_ + y_38_ + y_39_ + y_3_ + y_4_ + y_7_ + y_8_
 >= 1

_C10: y_30_ 

In [6]:
bristol_postcodes.assign(
    allocation = [str(assignment) for assignment in lscp1.cli2fac]
).explore('allocation',
         marker_kwds=dict(radius=10),
          cmap='Dark2'
         )

In [7]:
chosen = [len(clients) > 0 for clients in lscp1.fac2cli]

In [8]:
sum(chosen)

17

In [9]:
bristol_postcodes.assign(
    chosen=chosen
).explore('chosen',
        marker_kwds=dict(radius=10),
          cmap='Dark2'
         )

## Budget cuts incoming!

<div class="alert alert-warning">
    
Using the methods we discussed above, can you create a new MCLP problem object to cover as many Bristol post codes as possible with six facilities and a 5000 meter service radius? What facilities are not covered? 

</div>

In [10]:
mclp1 = spopt.locate.MCLP.from_geodataframe(
    gdf_demand=bristol_postcodes,
    gdf_fac=bristol_postcodes,
    service_radius = 5000,
    p_facilities=6,
    weights_cols='demand',
    demand_col="geometry", 
    facility_col="geometry",
).solve(pulp.COIN_CMD(msg=False))

In [11]:
bristol_postcodes.assign(
    allocation = [str(assignment) for assignment in mclp1.cli2fac]
).explore('allocation',
         marker_kwds=dict(radius=10),
          cmap='Dark2'
         )

# 

In [12]:
chosen = [len(clients) > 0 for clients in mclp1.fac2cli]
sum(chosen)

6

## Introduce capacity

Let's assume that each facility can only serve 1000 inhabitants. Let's create a column for this in our dataframe: 

```python
bristol_postcodes['capacity'] = 1_000
```

<div class="alert alert-warning">
    
Using the methods we discussed above and the `facility_capacity_col`, can you create a new LSCP problem object that includes this capacity constraint on facilities? How does it differ from the initial LSCP solution? 

</div>

In [13]:
bristol_postcodes['capacity'] = 1_000

In [14]:
# this may bug out on old versions of spopt!

In [15]:
lscp2 = spopt.locate.LSCP.from_geodataframe(
    gdf_demand=bristol_postcodes,
    gdf_fac=bristol_postcodes,
    service_radius = 5000,
    facility_capacity_col='capacity',
    demand_quantity_col='demand',
    demand_col="geometry",
    facility_col="geometry",
).solve(pulp.COIN_CMD(msg=False))

In [16]:
bristol_postcodes.assign(
    allocation = [str(assignment) for assignment in lscp1.cli2fac]
).explore('allocation',
         marker_kwds=dict(radius=10),
          cmap='Dark2'
         )

In [17]:
bristol_postcodes.assign(
    allocation = [str(assignment) for assignment in lscp2.cli2fac]
).explore('allocation',
         marker_kwds=dict(radius=10),
          cmap='Dark2'
         )

The assignments definitely change, primarily in the center city where the demand is highest. 

## MCLP is not PMedian

<div class="alert alert-warning">

Solve a P-Median problem for the same number of facilities (six) as our previous MCLP example. How do the MCLP/P-Median model solutions differ? 

</div>

In [18]:
pmedian = spopt.locate.PMedian.from_geodataframe(
    gdf_demand=bristol_postcodes,
    gdf_fac=bristol_postcodes,
    p_facilities=6,
    weights_cols='demand',
    demand_col="geometry", 
    facility_col="geometry",
).solve(pulp.COIN_CMD(msg=False))

## Challenge: Bristol is far from an isometric plane! 

<div class="alert alert-warning">
    
Using `spopt.locate.LSCP.from_cost_matrix()` function, can you create a new `LSCP` problem object to cover facilities using a 10 kilometer limit on the trip costs? How does this differ from your euclidean-based solution in the first question? 

</div>

In [19]:
cost_table = pandas.read_csv("../cost_table.csv")
cost_matrix = cost_table.pivot(index="destination_postcode", columns="origin_postcode", values="travel_cost")

In [20]:
cost_matrix

origin_postcode,BS1,BS10,BS11,BS12,BS13,BS14,BS15,BS16,BS17,BS18,...,BS41,BS43,BS48,BS49,BS5,BS6,BS7,BS73,BS8,BS9
destination_postcode,,,,,,,,,,,,,,,,,,,,,
BS1,0.0,16534.7,7448.0,4906.5,2110.4,8998.0,17166.8,11071.1,5097.5,13648.1,...,7206.6,4303.5,13478.3,22966.3,3539.9,4272.3,5858.1,21066.1,2488.8,4163.9
BS10,17665.2,0.0,15748.4,7790.1,17054.7,23942.4,19437.3,13341.5,16114.1,30719.2,...,27215.6,19247.8,24666.2,33617.9,15952.7,15288.9,12453.6,28165.1,15198.6,11374.4
BS11,7541.9,11710.0,0.0,7844.6,12443.6,17599.2,26673.8,20578.0,11858.8,22249.3,...,11242.6,11307.0,17514.3,26508.6,11697.5,11033.7,13273.6,35401.6,8360.3,4605.4
BS12,4840.7,7809.4,6901.1,0.0,7714.4,14602.1,14476.7,8381.0,6748.2,25758.7,...,11722.3,9907.5,17994.1,33076.3,6586.9,5923.0,5647.6,18376.0,6876.2,4425.4
BS13,2951.0,17236.3,12473.6,7996.3,0.0,7039.3,17868.4,11772.6,5799.0,11689.4,...,6934.3,2344.8,13206.0,22694.0,3128.3,4973.8,6559.7,21767.7,4028.9,6534.9
BS14,9458.3,23743.6,17950.6,14503.6,7474.9,0.0,18987.2,17116.9,12306.4,5919.5,...,11212.2,7434.2,17484.0,26971.9,8946.4,9867.5,13067.0,26332.6,11833.2,15773.9
BS15,17894.6,19516.5,30467.9,14480.6,17284.1,18352.1,0.0,7150.0,11164.5,20134.2,...,24066.7,19477.1,30338.4,48337.5,11983.0,11574.4,11867.9,9895.1,19559.0,18790.8
BS16,12684.7,14306.7,25258.1,9270.7,12074.2,16756.4,7256.6,0.0,4557.4,18538.5,...,18856.8,14267.3,25128.6,43127.6,6614.9,5613.3,5111.7,11155.8,14349.1,13581.0
BS17,5849.8,16009.0,11812.7,6762.4,5239.3,12126.9,11093.5,4716.0,0.0,16932.5,...,12021.9,7432.4,18293.6,27781.6,2840.4,2176.5,4178.2,15821.9,7514.2,7934.5


Let's make sure that the two are aligned before we use them:

In [21]:
assert (bristol_postcodes.outward == cost_matrix.index).all()
assert (bristol_postcodes.outward == cost_matrix.columns).all()

Since they are, we can proceed. If they weren't, we could ensure they are aligned using the following: 

```python
cost_matrix.loc[bristol_postcodes.outward.values, bristol_postcodes.outward.values]
```
as long as the indices are aligned together. Then, we can estimate the LSCP using the `from_cost_matrix` method: 

In [22]:
lscp_route = spopt.locate.LSCP.from_cost_matrix(
    cost_matrix.values, 
    service_radius = 10_000, 
).solve(pulp.COIN_CMD(msg=False))

Now, we can make a map of the route-based allocation, and you can read the euclidean-based allocation by hovering over each dot:

In [23]:
bristol_postcodes.assign(
    route_allocation = [str(assignment) for assignment in lscp_route.cli2fac],
    euclidean_allocation = [str(assignment) for assignment in lscp1.cli2fac]
).explore('route_allocation',
         marker_kwds=dict(radius=10),
          cmap='Dark2'
         )

The full table of assignment can be constructed directly:

In [24]:
pandas.concat(
    (
        pandas.Series(lscp_route.cli2fac, name='route-based', index=bristol_postcodes.outward), 
        pandas.Series(lscp1.cli2fac, name='straight line', index=bristol_postcodes.outward)
    ),
    axis=1
)

,route-based,straight line
outward,,
BS1,"[3, 31]","[41, 42]"
BS10,[3],[1]
BS11,[3],[42]
BS12,[3],[42]
BS13,"[3, 31]","[41, 42]"
BS14,[31],[30]
BS15,[23],[7]
BS16,"[3, 23]",[7]
BS17,"[3, 23, 31]",[7]


You could also create a map showing the places where the assignment is the same (or similar)

In [25]:
n_cover_in_common = [
    len(set(a1).intersection(set(a2)))
    for a1, a2 in zip(lscp_route.cli2fac, lscp1.cli2fac)
]

In [26]:
bristol_postcodes.assign(
    n_cover_in_common = n_cover_in_common
).explore('n_cover_in_common',
         marker_kwds=dict(radius=10),
          cmap='Dark2_r'
         )

Only the most remote of the locations have a similar coverage structure. Otherwise, the route-based solution is quite distinct from the euclidean one. This is why the `spopt` library can use arbitrary `cost_matrix` for solving location-allocation problems. 